In [25]:
import numpy as np, pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

## Write functions for parseing and printing blat files
def parse_psl(path,sr = 5,hd = None,rs=6, psl_cols = ['match','mismatch','rep_match', 
            'Ns','Q_gap_count','Q_gap_bases', 'T_gap_count',
            'T_gap_bases','T_strand', 'Q_name', 'Q_size', 'Q_start', 'Q_end', 'T_name', 
            'T_size', 'T_start', 'T_end','block_count', 'block_sizes', 'qStarts', 'tStarts']):
    psl_df = pd.read_csv(path,sep='\t',skiprows=sr,header=hd)
    psl_df.columns = psl_cols
    psl_df['Q_frac'] = np.round(psl_df['match']/psl_df['Q_size'],rs)
    psl_df.sort_values('Q_frac',ascending=False,inplace=True)
    psl_df.reset_index(drop=True,inplace=True)
    return psl_df

In [2]:
xl280path = '/home/croth/xl280genome.fasta'
h99path = '/home/croth/Documents/DirtyDozen/REF/FungiDB-46_CneoformansH99_Genome.fasta'

In [3]:
xl280 = [s for s in SeqIO.parse(xl280path,format='fasta')]
h99 = [s for s in SeqIO.parse(h99path,format='fasta')]

In [4]:
xl280_chrom7_qtl = (632254, 706135)

In [5]:
xl280_qtl_seq = [s.seq[xl280_chrom7_qtl[0]:xl280_chrom7_qtl[1]] 
                 for s in xl280 if s.id=='Chr07'][0]

In [6]:
assert len(xl280_qtl_seq) == np.min(np.diff(xl280_chrom7_qtl))

In [7]:
XL280_seqrec = SeqRecord(xl280_qtl_seq,id='XL280_QTL7')

In [30]:
save_path = '/home/croth/Desktop/xl280_amoeba_qtl_seq.fasta'
k = open(save_path,'w')
SeqIO.write(XL280_seqrec,format='fasta',handle=k)
k.close()

In [33]:
import subprocess,os

def blatem(database,query,pslout,blatloc = '/home/croth/./blat'):
    """Submit BLAT command to command line"""
    blatcommand = '%s %s %s %s'%(blatloc,query,database,pslout)
    process = subprocess.Popen(blatcommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    return (output,error)

In [34]:
psl_path = '/home/croth/Desktop/XL280_QTL7_to_H99.psl'

In [38]:
blatem(save_path,h99path,psl_path)

(b'Loaded 18916112 letters in 15 sequences\nSearched 73881 bases in 1 sequences\n',
 None)

In [39]:
qtl8_aligned = parse_psl(psl_path)

In [40]:
qtl8_aligned

,match,mismatch,rep_match,Ns,Q_gap_count,Q_gap_bases,T_gap_count,T_gap_bases,T_strand,Q_name,...,Q_end,T_name,T_size,T_start,T_end,block_count,block_sizes,qStarts,tStarts,Q_frac
0,60311,4723,0,0,395,8799,389,7702,-,XL280_QTL7,...,73871,CP003827.1,1398693,666895,739631,506,"7,282,230,301,255,5,1301,891,59,10,64,10,19,28...","10,20,308,543,851,1114,1120,2421,3319,3379,342...","666895,666906,667193,667423,667730,667994,6679...",0.816326
1,71,3,0,0,2,35,2,41,+,XL280_QTL7,...,29160,CP003821.1,1621675,911305,911420,3,"9,34,31,","29051,29071,29129,","911305,911327,911389,",0.000961
2,61,4,0,0,0,0,0,0,+,XL280_QTL7,...,36136,CP003826.1,1399503,10584,10649,1,"65,","36071,","10584,",0.000826
3,47,1,0,0,0,0,0,0,+,XL280_QTL7,...,21552,CP003834.1,24919,17623,17671,1,"48,","21504,","17623,",0.000636


In [41]:
27-19

8